In [ ]:
%%python -m pip install pip<24.1


DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


Импорт textract

In [ ]:
%pip install textract
import textract


DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [ ]:
import os
import textract

# Путь к папке с файлами
folder_path = "InputData"

# Поддерживаемые форматы
supported_extensions = ['.pdf', '.docx', '.txt', '.pptx', '.jpg', '.png', '.jpeg']

# Проходим по всем файлам в папке
for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)

    # Проверяем расширение файла
    if os.path.isfile(file_path) and any(file_path.lower().endswith(ext) for ext in supported_extensions):
        try:
            print(f"Обработка файла: {filename}...")

            # Извлекаем текст
            text = textract.process(file_path).decode('utf-8')

            # Сохраняем результат в TXT-файл с тем же именем
            output_path = os.path.join(folder_path, f"{os.path.splitext(filename)[0]}.txt")
            with open(output_path, 'w', encoding='utf-8') as f:
                f.write(text)

            print(f"Успешно! Результат сохранён в: {output_path}\n")

        except Exception as e:
            print(f"Ошибка при обработке {filename}: {str(e)}\n")

FileNotFoundError: [Errno 2] No such file or directory: 'путь/к/вашей/папке'

Загрузка BERT

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification, pipeline
import torch

# Загрузка предобученной модели и токенизатора
model_name = 'bert-base-multilingual-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=3)
"""
    #Инициализация мультиязычных моделей
    # Классификатор
    tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")
    classifier = BertForSequenceClassification.from_pretrained(
        "bert-base-multilingual-cased",
        num_labels=len(CLASS_NAMES)
    )
"""

# Универсальная модель для суммаризации
summarizer = pipeline(
    "summarization",
    model="facebook/mbart-large-50-many-to-many-mmt",  # Мультиязычная модель
    tokenizer="facebook/mbart-large-50-many-to-many-mmt"
)

Определение кластера текста с помощью BERT

In [ ]:
def DefineCluster(text):
  # Текст для классификации
  text = "Article: reasearch the effect of garbage on the human psyche"

  # Токенизация и подготовка входа
  inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)

  # Классификация
  with torch.no_grad():
      outputs = model(**inputs)
      predictions = torch.argmax(outputs.logits, dim=1)

  # Соответствие меток
  labels = ["email", "scientific_article", "resume"]
  print(f"Document classified as: {labels[predictions.item()]}")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Document classified as: scientific_article
